In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

In [44]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
celeri.plot_block_labels(segment, block, station, closure)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Build all linear operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)

# TODO: Update celeri.block_constraints to properly use block.
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Elastic operators
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
celeri.get_all_mesh_smoothing_matrices(meshes)

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

Calculating Okada partials for segments:   0%|          | 0/837 [00:00<?, ?it/s]

Calculating cutde partials for triangles:   0%|          | 0/1841 [00:00<?, ?it/s]

In [5]:
def plot_slip_distributions(mesh, slip_distribution_input, slip_distribution_estimated, suptitle_string):
    marker_size = 2.0
    plt.figure(figsize=(12, 7))

    plt.subplot(2, 2, 1)
    plt.title("input strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 2)
    plt.title("input dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_input[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 3)
    plt.title("estimated strike-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[0::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.subplot(2, 2, 4)
    plt.title("estimated dip-slip")
    plt.scatter(mesh.centroids[:, 0], mesh.centroids[:, 1], s=marker_size, c=slip_distribution_estimated[1::2], alpha=0.5)
    plt.colorbar()
    plt.gca().set_aspect("equal", adjustable="box")
    plt.xlim([230.0, 245.0])
    plt.ylim([37.5, 52.5])

    plt.suptitle(suptitle_string)
    plt.show()


def get_synthetic_displacements(mesh, tri_linear_operator):
    """
    Prescribe dip-slip in a Gaussian pattern.
    """
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - np.mean(mesh.centroids[:, 0])
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - np.mean(mesh.centroids[:, 1])

    # Hardcoded northern Cascadia example that Jack suggested.
    tri_centroid_to_mesh_lon = mesh.centroids[:, 0] - 234.5
    tri_centroid_to_mesh_lat = mesh.centroids[:, 1] - 48.5

    tri_centroid_to_mesh_centroid_distance = np.sqrt(tri_centroid_to_mesh_lon ** 2 + tri_centroid_to_mesh_lat ** 2)
    dip_slip_distribution = np.exp(-(tri_centroid_to_mesh_centroid_distance / 1.0) ** 2.0)
    slip_distribution = np.zeros(2 * dip_slip_distribution.size)
    slip_distribution[1::2] = dip_slip_distribution # Dip slip only
    synthetic_displacements = tri_linear_operator @ slip_distribution
    return slip_distribution, synthetic_displacements


# Shrink operators.tri_station so that there are no vertical displacments and no tensile slip
tde_matrix = copy.deepcopy(operators.tri_station)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[0], 3), axis=0)
tde_matrix = np.delete(tde_matrix, np.arange(2, tde_matrix.shape[1], 3), axis=1)

# Generate Guassian slip source and synthetic displacements
slip_distribution, synthetic_displacements = get_synthetic_displacements(meshes[0], tde_matrix)

In [6]:
# Slip estimation with np.linalg.lstsq
estimated_slip_distribution = np.linalg.lstsq(tde_matrix, synthetic_displacements, rcond=None)
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution[0], suptitle_string="np.linalg.lstsq - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Slip estimation with direct inverse no smoothing
estimated_slip_distribution = np.linalg.inv(tde_matrix.T @ tde_matrix) @ tde_matrix.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - no smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# Slip estimation with direct inverse and smoothing matrix
smoothing_matrix = meshes[0].smoothing_matrix.toarray()
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[0], 3), axis=0)
smoothing_matrix = np.delete(smoothing_matrix, np.arange(2, smoothing_matrix.shape[1], 3), axis=1)
smoothing_matrix = meshes[0].smoothing_weight * smoothing_matrix # Weight smoothing matrix
tde_and_smoothing_matrix = np.vstack((tde_matrix, smoothing_matrix))
synthetic_displacements_with_smoothing = np.hstack((synthetic_displacements, np.zeros(smoothing_matrix.shape[0]).T))
estimated_slip_distribution = np.linalg.inv(tde_and_smoothing_matrix.T @ tde_and_smoothing_matrix) @ tde_and_smoothing_matrix.T @ synthetic_displacements_with_smoothing
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_distribution, suptitle_string="Direct inverse - with smoothing")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# Slip estimation with truncated SVD
U, S, V_transpose = np.linalg.svd(tde_matrix)

In [10]:
# The general inverse is:
print("tde_matrix.shape :", tde_matrix.shape)
print("V_transpose.shape :", V_transpose.shape)
print("S.shape :", S.shape)
print("U.shape :", U.shape)
print("synthetic_displacements.shape :", synthetic_displacements.shape)

S_matrix = np.zeros_like(tde_matrix.T)
S_matrix_diag_index = np.arange(0, S.size)
S_matrix[S_matrix_diag_index, S_matrix_diag_index] = 1 / S
print("S_matrix.shape :", S_matrix.shape)

n_eigenvectors = 50
eigenvector_weights = np.linalg.pinv(tde_matrix @ V_transpose[:, 0:n_eigenvectors]) @ synthetic_displacements
estimated_slip_truncated_svd = V_transpose[:, 0:n_eigenvectors] @ eigenvector_weights
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Truncated SVD - Right eigenvectors only")

# Try full SVD
estimated_slip_full_svd = V_transpose @ S_matrix @ U.T @ synthetic_displacements
plot_slip_distributions(meshes[0], slip_distribution, estimated_slip_truncated_svd, suptitle_string="Full SVD")



# Try the singular value approach for old times sake:
# https://stackoverflow.com/questions/18452633/how-do-i-associate-which-singular-value-corresponds-to-what-entry

tde_matrix.shape : (3372, 3682)
V_transpose.shape : (3682, 3682)
S.shape : (3372,)
U.shape : (3372, 3372)
synthetic_displacements.shape : (3372,)
S_matrix.shape : (3682, 3372)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# Fall 2020 smoothing matrix SVD approach
# I think we only need the SVD of sparse smoothing matrix (I think there's scipy function for this)

# Sketching out the assembly of the block model system
The list here is in as of right now (09/19/21)...a work in progress

## The observation vector:    
| What | Symbolically | Variable name |
| - | - | - |
| GPS velocities | $\mathbf{v}$ | ```assembly.data.east_vel, assembly.data.north_vel``` |
| Slip rate constraints | $\dot{\mathbf{s}}$ | ```assembly.slip_rate_constraints``` |
| Rotation vector constraints | $\boldsymbol{\omega}$ | ```assembly.block_constraints``` |

## The state vector:    
| What | Symbolically | Variable name |
| - | - | - |
| Block rotation rates | $\boldsymbol{\Omega}$ | ```estimation.block_rotation_vectors``` |
| TDE slip | $\mathbf{t}$ | ```estimation.tri_slip_rates``` |
  
## Matrix operators:    
| What | Symbolically | Variable name |
| - | - | - |
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.block_rotation``` | 
| Rotation vector to GPS | $\boldsymbol{\Omega} \rightarrow \mathbf{v}$ | ```operators.global_float_block_rotation``` | 
| Rotation vector to segment slip rates| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_segment_block``` | 
| Segment slip rates to elastic deformation | $\dot{\mathbf{s}} \rightarrow \mathbf{v}$ | ```operators.okada_segment_station``` | 
| TDE slip to GPS | $\mathbf{t} \rightarrow \mathbf{v}$ | ```operators.tri_station``` |
| Rotation vector to slip rate constraints| $\boldsymbol{\Omega} \rightarrow \dot{\mathbf{s}}$ | ```operators.slip_rate_constraints``` |
| Rotation vector to Euler pole contraints | $\mathbf{I}$ | ```operators.block_motion_contraints``` |



In [46]:
print(operators.keys())
print(assembly.data.keys())

dict_keys(['block_rotation', 'global_float_block_rotation', 'block_motion_constraints', 'slip_rate_constraints', 'slip_rate_segment_block', 'block_strain_rate', 'mogi_station', 'okada_segment_station', 'tri_station'])
dict_keys(['n_stations', 'n_sar', 'east_vel', 'north_vel', 'up_vel', 'sar_line_of_sight_change_val', 'lon', 'lat', 'depth', 'x', 'y', 'z', 'block_label', 'n_block_constraints', 'block_constraints', 'n_slip_rate_constraints', 'slip_rate_constraints'])


In [51]:
# TODO: No stations on block with index 29!!!

data_vector = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)
operator = np.zeros((2 * len(station), 3 * len(block)))
operator = operators.block_rotation
block

,other1,other2,other3,other4,other5,other6,name,interior_lon,interior_lat,euler_lon,...,euler_lat_sig,rotation_rate,rotation_rate_sig,rotation_flag,apriori_flag,strain_rate,strain_rate_sig,strain_rate_flag,area_plate_carree,block_label
0,0,0,0,0,0,0,NorthAmerica,254.007,38.001,75.799,...,0.154,0.193,0.001,0,0,0,0,0,5.761291,0
1,0,0,0,0,0,0,ele56,245.241,39.099,65.697,...,2.084,0.222,0.013,0,0,0,0,0,3696.437226,1
2,0,0,0,0,0,0,newblock15,243.063,33.362,148.595,...,1.448,0.397,0.009,0,0,0,0,0,3.743816,2
3,0,0,0,0,0,0,newblock03,239.601,35.324,151.725,...,1.626,0.413,0.012,0,0,0,0,0,43.937039,3
4,0,0,0,0,0,0,newblock28,242.583,35.330,91.246,...,2.617,0.214,0.012,0,0,0,0,0,1.025052,4
5,0,0,0,0,0,0,newblock31,243.599,34.289,100.203,...,2.199,0.215,0.012,0,0,0,0,0,11003.506611,5
6,0,0,0,0,0,0,Restoftheworld,109.452,26.419,205.615,...,0.697,0.848,0.010,0,0,0,0,0,0.430015,6
7,0,0,0,0,0,0,ele37,241.612,38.380,69.286,...,2.214,0.219,0.013,0,0,0,0,0,14910.871277,7
8,0,0,0,0,0,0,ele67,238.023,48.795,62.208,...,0.831,0.232,0.013,0,0,0,0,0,11.864791,8
9,0,0,0,0,0,0,ele40,235.313,41.256,61.471,...,0.588,0.238,0.013,0,0,0,0,0,36.259160,9


In [29]:
state_vector = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 

LinAlgError: Singular matrix